# **NETFLIX RECOMMENDATION SYSTEM**
This is a content based recommendation system.

I have uploaded another notebook where I have created class and funtions to automate the process,aka, you won't have to run each cell, just call the automate() function and it will create the recommendation engine for you.


# **LOGIC AND FLOW**

1. Clean the data and seperate the data into 2 different datasets(Movie and TV Shows)
2. Remove stopwords(the,a,and,etc.) and special characters(!,&,%,etc.) from the text as they might skew the analysis and hinder the recommendation system's output.
3. Vectorize the desired columns of both the datasets, these will be used for calculating cosine simlarity. Vectorizing is similar to one hot encoding where text is split into tokens/words/bag-of-words and values are alotted. 1 if the word/token/bag-of-words is present else 0.
4. These vecotized columns are concatinated into a single dataset where rows are movie or tv shows and columns are  word/token/bag-of-words.
5. Cosine Similarity of the two vectorized datasets is calculated. Values are between 0 and 1. Higher the value of Movie 1 and Movie 2, implies they are smililar and movie 2 shold be recommended to someone who likes movie 1.
6. A recommendation function is created, where given an input movie/tv show, the cosine values of other movies/tv shows is compared and the top 5 movies/tv shows with highest cosine values are recommended.


# **COSINE SIMILARITY**
Takes the concatenated vectorized datasets as input and calculates cosine_similarity. Cosine_similarity calculates angle between 2 vectors, here each vector is the vectorized data of a movie.

$$
\text{Cosine Similarity (A, B)} = \frac{A \cdot B}{\|A\| \|B\|}
$$

Where A and B are of the form: [0,1,1,0,0,........] and A.B is the dot product of the vectors, and \|A\| \|B\| is the product of the magnitude of these vectors.

The resulting output is square matrix of the size = number of movies/tv show. Each cell will have a value between 0 and 1, where 0 means no co-relation/similarity between the 2 movies and 1 means both movies are the same.

For example, if the cosine value at matrix location [7,50] is 0.854 means the simialiry between Movie Number 8 and Movie Number 51 is high.

The higher the value, the more similar/co-related the movies/tv shows and vice versa.

In [35]:
#installing required library
%pip install neattext

In [36]:
#importing required libraris
import pandas as pd
import numpy as np
import neattext.functions as nfx
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#to ignore warning prompts
import warnings
warnings.filterwarnings('ignore')

In [37]:
#reading the dataset
df=pd.read_csv('netflix_titles.csv')
print(df.shape)
df.head(4)

(8807, 12)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."


In [45]:
2446/2676


0.914050822122571

In [6]:
#renaming column
df.rename(columns={'listed_in':'genre'},inplace=True)

In [7]:
# Check how many types of films are included
df.type.value_counts()

,count
type,
Movie,6131
TV Show,2676


# **PREPARING DATA FOR VECTORIZATION**

## **CLEANING MOVIE DATA**

In [8]:
# stroing movies in a different dataset and resetting index
df_movie=df[df['type']=='Movie'].copy()
df_movie.reset_index(drop=True,inplace=True)
print(df_movie.shape)
df_movie.head(4)

(6131, 12)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genre,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...
2,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
3,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...


In [9]:
#checking null values in movie dataset
df_movie.isna().sum()

,0
show_id,0
type,0
title,0
director,188
cast,475
country,440
date_added,0
release_year,0
rating,2
duration,3


In [10]:
# Dropping rows null values since
df_movie.dropna(inplace= True)
df_movie.reset_index(drop=True,inplace=True)

In [11]:
#getting the desired columns for vectorization(explained further down in the code)
movies=df_movie[['title','director','cast','country','genre','rating']].copy()
movies.head(4)

,title,director,cast,country,genre,rating
0,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","Dramas, Independent Movies, International Movies",TV-MA
1,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"Comedies, Dramas",PG-13
2,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic","Dramas, International Movies",TV-MA
3,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...",India,"Comedies, International Movies, Romantic Movies",TV-14


In [12]:
#getting few statistical metrics about the dataset columns
movies.describe().T

,count,unique,top,freq
title,5185,5185,Sankofa,1
director,5185,3828,"Raúl Campos, Jan Suter",18
cast,5185,5061,Samuel West,10
country,5185,594,United States,1819
genre,5185,268,"Dramas, International Movies",336
rating,5185,14,TV-MA,1741


### CREATING FUNCTION TO REMOVE STOPWORDS AND SPECIAL CHARACTER

In [13]:
#creating a function to remove stopwords and special characters
def remove_stop_char(data,column,sw=True): #sw=stopwords
  if sw:
    #Removing stopwords(the, and, etc.) since they carry little to no semantic meaning and can skew the analysis and affect the accuracy
    data[column]=data[column].apply(nfx.remove_stopwords)
  else:
    #removing special characters(%, #, etc) since they don't contribute to the meaning of the text
    data[column]=data[column].apply(nfx.remove_special_characters)

In [14]:
#calling the function for the required columns to remove stopwords and special characters
remove_stop_char(movies,'director')
remove_stop_char(movies,'cast')
remove_stop_char(movies,'country')
remove_stop_char(movies,'genre')

remove_stop_char(movies,'country',False)

movies.head(4)

,title,director,cast,country,genre,rating
0,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...",United States Ghana Burkina Faso United Kingdo...,"Dramas, Independent Movies, International Movies",TV-MA
1,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"Comedies, Dramas",PG-13
2,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...",Germany Czech Republic,"Dramas, International Movies",TV-MA
3,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...",India,"Comedies, International Movies, Romantic Movies",TV-14


## **CLEANING TV SHOW DATA**

In [15]:
# stroing tv shows in a different dataset and resetting index
df_tv=df[df.type=='TV Show'].copy()
df_tv.reset_index(drop=True, inplace=True)
print(df_tv.shape)
df_tv.head(4)

(2676, 12)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genre,description
0,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
1,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
2,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
3,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [16]:
#checking null values in tv dataset
df_tv.isna().sum()

,0
show_id,0
type,0
title,0
director,2446
cast,350
country,391
date_added,10
release_year,0
rating,2
duration,0


In [17]:
# filling NaN manually in the director column so pandas can treat it as a non-null value
# need to do this since most of the valuesin director columns are Null, dropping them means we will have hardly any rows/tv shows to work with
df_tv['director'].fillna('NaN', inplace= True)

# Dropping null values
df_tv.dropna(inplace= True)
df_tv.reset_index(drop=True,inplace=True)

In [18]:
#getting the desired columns for vectorization(explained further down in the code)
tv=df_tv[['title','director','cast','country','genre','rating']].copy()
tv.head(4)

,title,director,cast,country,genre,rating
0,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"International TV Shows, TV Dramas, TV Mysteries",TV-MA
1,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"International TV Shows, Romantic TV Shows, TV ...",TV-MA
2,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"British TV Shows, Reality TV",TV-14
3,Dear White People,NaN,"Logan Browning, Brandon P. Bell, DeRon Horton,...",United States,"TV Comedies, TV Dramas",TV-MA


In [19]:
#getting few statistical metrics about the dataset
tv.describe().T

,count,unique,top,freq
title,2013,2013,Blood & Water,1
director,2013,142,NaN,1866
cast,2013,1980,David Attenborough,14
country,2013,184,United States,618
genre,2013,219,Kids' TV,161
rating,2013,9,TV-MA,881


In [20]:
#calling the function for the required columns to remove stopwords and special characters
remove_stop_char(tv,'cast')
remove_stop_char(tv,'country')
remove_stop_char(tv,'genre')

remove_stop_char(tv,'country',False)

# **VECTORIZATION**

## **VECTORZING MOVIE DATASET**

### **CREATING A FUNCTION TO VECTORIZE THE TEXT**

In [21]:
#Creating af function to vectorize the text data
def vectorization(data,column,token=True): #token=True imples ','(comma) will be the delimiter, else default(blank space)
  if token:
    #all the characters between ','(comma) will be considered one tokken, aka, ','(comma) works as delimiter
    #example: for CAST column, each cast name is divided by a comma, hence each cast is a token, and if the movie has that cast it will be marked 1, else 0, aka, binary.
    countVector = CountVectorizer(binary=True, tokenizer=lambda x:x.split(','))
    return countVector.fit_transform(data[column]).toarray()
  else:
    #here by default, blank space is considered a delimiter
    countVector = CountVectorizer(binary=True)
    return countVector.fit_transform(data[column]).toarray()

In [22]:
#calling the function to vectorize the desired column
country = vectorization(movies,'country',False)
director = vectorization(movies,'director')
cast = vectorization(movies,'cast')
genre = vectorization(movies,'genre')

In [23]:
#converting vectorised arrays to dataset
binary_director=pd.DataFrame(director)
binary_cast=pd.DataFrame(cast)
binary_country=pd.DataFrame(country)
binary_genre=pd.DataFrame(genre)

In [24]:
# Concating the datasets vertically, aka horizontal join
movie_binary = pd.concat([binary_director,binary_cast,binary_country,binary_genre],axis=1, ignore_index=True)
movie_binary

,0,1,2,3,4,5,6,7,8,9,...,30805,30806,30807,30808,30809,30810,30811,30812,30813,30814
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5180,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5181,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5182,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5183,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
#calculating the cosine_similarity, which means it compares the values in each row(each movie) against each other and gives a similarity score from 0 to 1
#For Example, if 2 movies have same director and similar cast they will get a higher similarity score and vice versa
#Another Example, if 2 movies have similar description and genre, they will get a higher similarity score and vice versa
movie_sim=cosine_similarity(movie_binary)
movie_sim

array([[1.        , 0.1118034 , 0.16269784, ..., 0.12909944, 0.11952286,
        0.12403473],
       [0.1118034 , 1.        , 0.        , ..., 0.21650635, 0.13363062,
        0.        ],
       [0.16269784, 0.        , 1.        , ..., 0.        , 0.        ,
        0.13453456],
       ...,
       [0.12909944, 0.21650635, 0.        , ..., 1.        , 0.15430335,
        0.        ],
       [0.11952286, 0.13363062, 0.        , ..., 0.15430335, 1.        ,
        0.        ],
       [0.12403473, 0.        , 0.13453456, ..., 0.        , 0.        ,
        1.        ]])

In [26]:
#shape is number of movies X number of movies
movie_sim.shape

(5185, 5185)

## **VECTORIZATING TV SHOW DATA**

In [27]:
#calling the function to vectorize the desired column
tv_country = vectorization(df_tv,'country',False)
tv_cast = vectorization(df_tv,'cast')
tv_genre = vectorization(df_tv,'genre')

In [28]:
#converting vectorised arrays to dataset
tv_binary_cast=pd.DataFrame(tv_cast)
tv_binary_country=pd.DataFrame(tv_country)
tv_binary_genre=pd.DataFrame(tv_genre)

In [29]:
# Concating the datasets vertically, aka horizontal join
tv_binary = pd.concat([tv_binary_cast,  tv_binary_country, tv_binary_genre], axis=1,ignore_index=True)
tv_binary

,0,1,2,3,4,5,6,7,8,9,...,14061,14062,14063,14064,14065,14066,14067,14068,14069,14070
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2009,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2010,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2011,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
#calculating the cosine_similarity
tv_sim = cosine_similarity(tv_binary)
tv_sim

array([[1.        , 0.05892557, 0.        , ..., 0.08908708, 0.05455447,
        0.1132277 ],
       [0.05892557, 1.        , 0.        , ..., 0.06299408, 0.        ,
        0.16012815],
       [0.        , 0.        , 1.        , ..., 0.        , 0.09449112,
        0.        ],
       ...,
       [0.08908708, 0.06299408, 0.        , ..., 1.        , 0.        ,
        0.12104551],
       [0.05455447, 0.        , 0.09449112, ..., 0.        , 1.        ,
        0.        ],
       [0.1132277 , 0.16012815, 0.        , ..., 0.12104551, 0.        ,
        1.        ]])

# **CREATING RECOMENDATION ENGINE(FUNCTION)**

In [31]:
#creating function for recommendation, this function will be our recommendation engine
def recommedation(title): # title = user input

  if title in df_movie.title.values: #checking if title is in movie dataset

    #getting the index value of the row where title is present, .item() converts it into scalar
    index=df_movie[df_movie.title == title].index.item()

    #finding row(index/movie) in the cosine similarity data and storing the similarity score of the selected title(movie) against all other movies as a dictionary
    #here key will be the index of the movies and value will be the similarity score
    scores=dict(enumerate(movie_sim[index]))
    sorted_scores=dict(sorted(scores.items(),key=lambda x:x[1],reverse=True))#sorting the dictionary in descending order of the similarity score(highest one first)

    selected_movies_index=[id for id, scores in sorted_scores.items()] #storing all keys(index/row/movie) in a list
    selected_movies_score=[scores for id, scores in sorted_scores.items()]#storing all values(similarity score) in another list

    #getting all rows from df_movie dataset using the sorted keys, aka movie with the highest similarity score will be on top
    recommend_movies=df_movie.iloc[selected_movies_index]
    recommend_movies['similarity'] = selected_movies_score #adding a column with similarity score using sorted values

    movie_recommend = recommend_movies.reset_index(drop=True)
    return movie_recommend[1:6] #returing top 5 movies, first row is skipped since it will be the selected movie itself(similarity score =1)

  #perforiming all the above steps for tv show dataset
  elif title in df_tv['title'].values:

    index=df_tv[df_tv.title == title].index.item()
    scores=dict(enumerate(tv_sim[index]))
    sorted_scores=dict(sorted(scores.items(),key=lambda x:x[1],reverse=True))

    selected_tv_index=[id for id, scores in sorted_scores.items()]
    selected_tv_score=[scores for id, scores in sorted_scores.items()]

    recommend_tv=df_tv.iloc[selected_tv_index]
    recommend_tv['similarity'] = selected_tv_score

    tv_recommend = recommend_tv.reset_index(drop=True)
    return tv_recommend[1:6]

  else:
    print('Title not found')

In [32]:
#calling the function for a movie to get recommendation
recommedation("Child's Play")

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genre,description,similarity
1,s6416,Movie,Candyman,Bernard Rose,"Virginia Madsen, Tony Todd, Xander Berkeley, K...","United States, United Kingdom","October 1, 2019",1992,R,99 min,"Cult Movies, Horror Movies",Grad student Helen Lyle unintentionally summon...,0.276026
2,s8238,Movie,The Car,Elliot Silverstein,"James Brolin, Kathleen Lloyd, John Marley, R.G...",United States,"June 1, 2020",1977,PG,96 min,"Cult Movies, Horror Movies","In his small Southwestern town, sheriff Wade P...",0.276026
3,s797,Movie,Hostel: Part III,Scott Spiegel,"Kip Pardue, Brian Hallisay, John Hensley, Sara...",United States,"June 2, 2021",2011,R,88 min,"Cult Movies, Horror Movies",In this installment in the popular horror fran...,0.266667
4,s4525,Movie,Tales From the Hood 2,"Rusty Cundieff, Darin Scott","Keith David, Bryan Batt, Alexandria Deberry, B...",United States,"October 10, 2018",2018,R,110 min,"Cult Movies, Horror Movies, Independent Movies",Buckle up for an anthology of socially conscio...,0.266667
5,s6545,Movie,Cult of Chucky,Don Mancini,"Fiona Dourif, Michael Therriault, Adam Hurtig,...",United States,"October 3, 2017",2017,R,90 min,Horror Movies,Following a string of murders in the asylum wh...,0.266667


In [33]:
#calling the function for a tv show to get recommendation
recommedation('Kota Factory')

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genre,description,similarity
1,s3294,TV Show,Little Things,NaN,"Dhruv Sehgal, Mithila Palkar",India,"November 9, 2019",2019,TV-MA,3 Seasons,"International TV Shows, Romantic TV Shows, TV ...",A cohabiting couple in their 20s navigate the ...,0.471405
2,s7873,TV Show,Rishta.com,NaN,"Shruti Seth, Kavi Shastri, Siddhant Karnick, K...",India,"March 15, 2018",2010,TV-14,1 Season,"International TV Shows, Romantic TV Shows, TV ...",Partners at an Indian matrimonial agency face ...,0.408248
3,s7454,TV Show,Midnight Misadventures With Mallika Dua,NaN,Mallika Dua,India,"April 1, 2019",2018,TV-14,1 Season,"International TV Shows, Stand-Up Comedy & Talk...","In this talk show, comedian Mallika Dua serves...",0.387298
4,s8776,TV Show,Yeh Meri Family,NaN,"Vishesh Bansal, Mona Singh, Akarsh Khurana, Ah...",India,"August 31, 2018",2018,TV-PG,1 Season,"International TV Shows, TV Comedies","In the summer of 1998, middle child Harshu bal...",0.365148
5,s1590,TV Show,Bhaag Beanie Bhaag,NaN,"Swara Bhasker, Dolly Singh, Ravi Patel, Varun ...",India,"December 4, 2020",2020,TV-MA,1 Season,"International TV Shows, Romantic TV Shows, TV ...","Facing disapproving parents, a knotty love lif...",0.365148


In [34]:
#calling the function for a movie to get recommendation
recommedation('After')

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genre,description,similarity
1,s1502,Movie,After We Collided,Roger Kumble,"Josephine Langford, Hero Fiennes Tiffin, Dylan...",United States,"December 22, 2020",2020,R,105 min,"Dramas, Romantic Movies","Tessa fell hard and fast for Hardin, but after...",0.726722
2,s5512,Movie,Rodney King,Spike Lee,Roger Guenveur Smith,United States,"April 28, 2017",2017,TV-MA,53 min,Dramas,Roger Guenveur Smith gives voice to the man at...,0.335410
3,s1331,Movie,The World We Make,Brian Baugh,"Caleb Castille, Rose Reid, Kevin Sizemore, Gre...",United States,"February 10, 2021",2019,PG,108 min,"Dramas, Romantic Movies",A teenage equestrian and a local football play...,0.333333
4,s5688,Movie,Blue Jay,Alex Lehmann,"Sarah Paulson, Mark Duplass, Clu Gulager",United States,"December 6, 2016",2016,TV-MA,81 min,"Dramas, Independent Movies, Romantic Movies",Two former high school sweethearts unexpectedl...,0.333333
5,s6391,Movie,Burlesque,Steve Antin,"Cher, Christina Aguilera, Alan Cumming, Eric D...",United States,"December 16, 2019",2010,PG-13,119 min,"Dramas, Romantic Movies","After leaving Iowa with stars in her eyes, Ali...",0.322749
